# Libraries

In [1]:
import cv2
import itertools
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle
import re

from datetime import datetime
from itertools import cycle
from PIL import Image
from time import time
from tqdm import tqdm_notebook as tqdm
from zipfile import ZipFile

In [2]:
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.metrics import classification_report
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.ensemble import RandomForestClassifier

In [3]:
from tensorflow.python.keras import Input, Model
from tensorflow.python.keras import layers
from tensorflow.python.keras.utils import to_categorical

/opt/ds3/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Constants

In [4]:
# FILENAME = r'C:/Users/leohe/Documents/ITC/HIVE/Challenge/sign_language/HSL_images.zip'
FILENAME = r'/data/sign_language/HSL_images.zip'


# LABEL = {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9,
#          'K':10, 'L':11, 'M':12, 'N':13, 'O':14, 'P':15, 'Q':16, 'R':17, 'S':18, 'T':19,
#          'U':20, 'V':21, 'W':22, 'X':23, 'Y':24, 'Z':25, 
#          'del':26, 'nothing':27, 'space':28}

LABEL = {'alef': 0, 'bet': 1, 'gimel': 2, 'daled': 3, 
         'hey': 4, 'vav': 5, 'zayin': 6, 'het': 7, 'tet': 8, 
         'yud': 9, 'haf': 10, 'lamed': 11, 'mem': 12, 'nun': 13, 
         'samech': 14, 'ayin': 15, 'pey': 16, 'tzadik': 17, 
         'kuf': 18, 'reish': 19, 'shin': 20, 'taf': 21, 
         'space': 22, 'empty': 23}


NUM_REGEX = r"[0-9]+"

# SAMPLE_SIZE = 30
N_TRAIN = 200 * 24 * 10
N_TEST  = 200 * 24 * 1 #87000 // SAMPLE_SIZE# total number of images in the training set
IMG_SIZE = 200 # original image size, to be resized
CHANNEL = 3

# Functions

In [5]:
def idos_filter(img, *options):
    return img

- loading the images into a np.array from inside the zip file
- loading and translating the labels

In [6]:
train_count = 0
x_train  = np.zeros((N_TRAIN, IMG_SIZE, IMG_SIZE, CHANNEL), dtype='float16')
y_train = np.zeros((N_TRAIN, 1), dtype='uint8')

test_count = 0
x_test  = np.zeros((N_TEST, IMG_SIZE, IMG_SIZE, CHANNEL), dtype='float16')
y_test = np.zeros((N_TEST, 1), dtype='uint8')

with ZipFile(FILENAME) as archive:
    for entry in tqdm(archive.infolist()):
        name = str(entry).split("'")[1].replace('/', ' ').split()
        if len(name) == 3:
            if int(re.findall(NUM_REGEX, name[2])[0]) <= 200:
                with archive.open(entry) as file:
                    img = np.array(Image.open(file))
                x_test[test_count] = idos_filter(img) / 255
                y_test[test_count] = LABEL[name[1]]
                test_count += 1
            elif 200 < int(re.findall(NUM_REGEX, name[2])[0]):
                with archive.open(entry) as file:
                    img = np.array(Image.open(file))
                x_train[train_count] = idos_filter(img) / 255
                y_train[train_count] = LABEL[name[1]]
                train_count += 1

In [7]:
inputs = layers.Input(shape=x_train.shape[1:])
f = layers.Conv2D(64, (3, 3), kernel_initializer='glorot_normal')(inputs)
f = layers.Conv2D(64, (3, 3), kernel_initializer='glorot_normal', activation='relu')(f)
f = layers.BatchNormalization()(f)
f = layers.MaxPool2D()(f)
f = layers.Conv2D(16, (3, 3), kernel_initializer='glorot_normal', activation='relu')(f)
f = layers.Conv2D(16, (3, 3), kernel_initializer='glorot_normal', activation='relu')(f)
f = layers.BatchNormalization()(f)
f = layers.MaxPool2D()(f)
f = layers.Conv2D( 4, (3, 3), kernel_initializer='glorot_normal', activation='relu')(f)
f = layers.Conv2D( 4, (3, 3), kernel_initializer='glorot_normal', activation='relu')(f)
f = layers.BatchNormalization()(f)
f = layers.MaxPool2D()(f)
f = layers.Flatten()(f)
f = layers.Dense(400, activation='relu', kernel_initializer='glorot_normal')(f)
f = layers.Dropout(0.2)(f)
f = layers.Dense(200, activation='relu', kernel_initializer='glorot_normal')(f)
f = layers.Dropout(0.2)(f)
f = layers.Dense(100, activation='relu', kernel_initializer='glorot_normal')(f)
f = layers.Dropout(0.2)(f)
f = layers.Dense(100, activation='relu', kernel_initializer='glorot_normal')(f)
f = layers.Dropout(0.2)(f)
f = layers.Dense( 50, activation='relu', kernel_initializer='glorot_normal')(f)
f = layers.Dropout(0.2)(f)
outputs = layers.Dense(len(np.unique(y_train)), activation='softmax')(f)
model = Model(inputs=inputs, outputs=outputs)
print(model.summary())
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 200, 200, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 198, 198, 64)      1792      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 196, 196, 64)      36928     
_________________________________________________________________
batch_normalization_1 (Batch (None, 196, 196, 64)      256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 98, 98, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 96, 96, 16)        9232      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 94, 94, 16)        2320      
__________

In [8]:
history = model.fit(x= x_train, 
                    y= to_categorical(y_train, len(np.unique(y_train))), 
                    batch_size= 128, 
                    epochs= 2)

Epoch 1/2
48000/48000 [==============================] - 627s 13ms/step - loss: 1.9572 - acc: 0.3955
Epoch 2/2
48000/48000 [==============================] - 488s 10ms/step - loss: 0.4958 - acc: 0.8379


In [9]:
loss, acc = model.evaluate(x=x_test, y=to_categorical(y_test, len(np.unique(y_test))))
print(loss, acc)
probabilities = model.predict(x=x_test)
y_pred = np.argmax(probabilities, axis=-1)

4800/4800 [==============================] - 16s 3ms/step
2.5002824674298365 0.5108333333333334


In [10]:
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.27      0.35      0.31       200
          1       0.64      0.56      0.60       200
          2       0.37      0.46      0.41       200
          3       0.79      0.59      0.68       200
          4       0.48      0.67      0.56       200
          5       0.45      0.63      0.52       200
          6       0.77      0.70      0.73       200
          7       0.28      0.43      0.34       200
          8       0.61      0.62      0.62       200
          9       0.26      0.47      0.34       200
         10       0.55      0.53      0.54       200
         11       0.79      0.72      0.76       200
         12       0.65      0.14      0.23       200
         13       0.55      0.06      0.10       200
         14       0.13      0.09      0.10       200
         15       0.59      0.79      0.68       200
         16       0.44      0.43      0.44       200
         17       0.81      0.90      0.85   